In [200]:
import pandas as pd

### Import des données

In [201]:
UO = pd.read_excel(r'C:\Users\sgasmi\Desktop\Dev\UO\Copie de structure_20202412.xlsx') # REF UO ETL
UO_P = pd.read_excel(r'C:\Users\sgasmi\Desktop\Export Python\UO_Org.xlsx')

### Ajout du niveau

In [202]:
UO["Level"]= UO["arborescence"].str.count('>')
UO["Level"]= UO["Level"].apply(lambda x: "L" + str(int((x/2)+1)) if x !=0 else "L1")


### Ajout de l'UO mère

In [203]:
UO_mere = UO[["Id","shortname"]]

In [204]:
UO = UO.merge(UO_mere, left_on="id parent", right_on="Id", how="left")
UO = UO.drop("Id_y", axis=1)
UO = UO.rename(columns={"shortname_y": "UO mère", "shortname_x": "shortname"})
UO.loc[UO["UO mère"].isna()==True, "UO mère"] = "ROOT"

### Filtre sur les UO de niveau 3 et SPiIC pour People Net et ref ETL UO

In [205]:
UO_P = UO_P.loc[UO_P["Code type d'unité de tr."].isin(["L1", "L2", "L3"])]
UO = UO.loc[UO["Level"].isin(["L1", "L2", "L3"])]

### Détection des UO à créer

In [206]:
UO_P["A créer"] = 0
UO_P.loc[UO_P["Nom de l'unité de travail"].isin(UO["shortname"])==False, "A créer"] = 1
Create_UO = UO_P[UO_P["A créer"]==1]

### Détection de parents différents

In [207]:
UO_P = UO_P.merge(UO[["shortname", "Level", "UO mère"]], left_on="Nom de l'unité de travail", right_on="shortname", how="left")
UO_P = UO_P.rename(columns={"UO mère_x" : "UO mère", "UO mère_y" : "UO mère ETL"})
UO_P["Ecart_parent"] = UO_P.apply(lambda x: "Oui" if x["UO mère"]!=x["UO mère ETL"] else "", axis=1)
Update_mere = UO_P.loc[(UO_P["Ecart_parent"]=="Oui") & (UO_P["A créer"]==0)]

### Détection niveau différent

In [208]:
UO_P["Ecart niveau"] = UO_P.apply(lambda x: "Oui" if x["Code type d'unité de tr."] != x["Level"] else "", axis=1)
Update_level=UO_P[(UO_P["Ecart niveau"]=="Oui") & (UO_P["A créer"]==0)]

### Détection des UO à supprimer

In [209]:
UO["A supprimer"] = 0
UO.loc[UO["shortname"].isin(UO_P["Nom de l'unité de travail"])==False, "A supprimer"] = 1
Delete_UO = UO.loc[UO["A supprimer"]==1]

### Export du résultat

In [210]:
with pd.ExcelWriter(r'C:\Users\sgasmi\Desktop\Export Python\Update UO.xlsx') as writer:
    if len(Create_UO)>0:
        Create_UO.to_excel(writer, sheet_name='UO à ajouter', index=False)
    if len(Update_mere)>0:    
        Update_mere.to_excel(writer, sheet_name='UO mère à modifier', index=False)
    if len(Update_level)>0:
        Update_level.to_excel(writer, sheet_name='Niveau UO à changer', index=False)
    if len(Delete_UO)>0:
        Delete_UO.to_excel(writer, sheet_name='UO à supprimer', index=False)